SI BESOIN ANALYSE TWEETS 16M RAWS HUGGING FACE

In [2]:
from datasets import load_dataset

ds = load_dataset("ckandemir/bitcoin_tweets_sentiment_kaggle")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Generating eval spl

In [5]:
small = ds["train"].select(range(1000))  # 1000 premières lignes
df_small = small.to_pandas()
df_small.head()


,Date,text,Sentiment
0,2019-06-12,Are you interested in Bitcoin mining for FREE!...,Positive
1,2019-07-08,"QA Analyst - IT Services Company ( WESTBURY, U...",Positive
2,2019-05-22,🌍22/05/2019🌍\nA Abakus a qual a NovaDAX perten...,Negative
3,2019-05-16,@Quark_Chain #QKC #Blockchain #BTC #ETH #block...,Negative
4,2018-07-21,"July 21, 2018 04:00 AM EDT\nLast 4 hours, BTC ...",Positive


NETTOYAGE DES DATASETS

In [2]:
import pandas as pd
import numpy as np

from pathlib import Path


chargement des datasets

In [3]:
# Chemins
BASE_PATH = Path("../data/brutes/sentiment")

fear_greed_path = BASE_PATH / "fear_and_greed_index.csv"
news_sentiment_path = BASE_PATH / "bitcoin-news-sentiments.csv"
news_sentiment_21_24_path = BASE_PATH / "bitcoin_sentiments_21_24.csv"

# Load
df_fear = pd.read_csv(fear_greed_path)
df_news = pd.read_csv(news_sentiment_path)
df_news_21_24 = pd.read_csv(news_sentiment_21_24_path)

# Quick check
print(df_fear.head())
print(df_news.head())
print(df_news_21_24.head())


    fng_value  fng_classification          date
0  07-09-2022                  24  Extreme Fear
1  06-09-2022                  22  Extreme Fear
2  05-09-2022                  23  Extreme Fear
3  04-09-2022                  20  Extreme Fear
4  03-09-2022                  21  Extreme Fear
                                           Headlines        Date Sentiments  \
0  Price analysis 3/15: BTC, ETH, BNB, SOL, XRP, ...  2024-03-15    neutral   
1  Bitcoin drops 9% from its ATH as the market sh...  2024-03-15   negative   
2  UK court rules Craig Wright not Nakamoto, endi...  2024-03-15    neutral   
3  Wealth manager Cetera adds spot BTC ETFs to it...  2024-03-15    neutral   
4  Who is ‘Mr. 100’? Mysterious Bitcoin whale bec...  2024-03-15    neutral   

                                   Source  
0  https://cointelegraph.com/tags/bitcoin  
1  https://cointelegraph.com/tags/bitcoin  
2  https://cointelegraph.com/tags/bitcoin  
3  https://cointelegraph.com/tags/bitcoin  
4  https://cointe

Normalisation des timestamps (UTC)

In [ ]:
#Fear and Greed Index
# Renommer les colonnes selon leur vrai contenu
# fng_value contient le score numérique, fng_classification le texte de classification.
df_fear = df_fear.rename(columns={
    "fng_value": "fear_greed_index",
    "fng_classification": "classification_text",
})


In [ ]:
# Convertir la vraie date (format jour-mois-année)
# Utiliser dayfirst=True pour jour-mois-année et errors='coerce' pour attraper les valeurs invalides.
df_fear["date"] = pd.to_datetime(df_fear["date"], dayfirst=True, utc=True, errors='coerce')

# Garder uniquement ce qui est utile
df_fear = df_fear[["date", "fear_greed_index"]]

# S'assurer que le score est numérique
df_fear["fear_greed_index"] = pd.to_numeric(df_fear["fear_greed_index"], errors='coerce')

# Supprimer les lignes dont la date n'a pas pu être parsée
df_fear = df_fear.dropna(subset=["date"])

# Trier
df_fear = df_fear.sort_values("date")

df_fear.head()

KeyError: 'date'

In [14]:
#News FinGPT
df_news["Date"] = pd.to_datetime(df_news["Date"], utc=True)
df_news = df_news.sort_values("Date")
sentiment_map = {
    "positive": 1,
    "neutral": 0,
    "negative": -1
}

df_news["sentiment_num"] = df_news["sentiment"].map(sentiment_map)


KeyError: 'sentiment'